In [1]:
# import required packages
import cv2
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [2]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [6]:
# Preprocess all train images
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

# Preprocess all validation images
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [7]:
# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

cv2.ocl.setUseOpenCL(False)

In [8]:
emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001, decay=1e-6), metrics=['accuracy'])

# Train the neural network/model
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=7178 // 64)

Epoch 1/50
448/448 [==============================] - 741s 2s/step - loss: 1.8060 - accuracy: 0.2559 - val_loss: 1.7369 - val_accuracy: 0.3345
Epoch 2/50
448/448 [==============================] - 511s 1s/step - loss: 1.6284 - accuracy: 0.3655 - val_loss: 1.5278 - val_accuracy: 0.4188
Epoch 3/50
448/448 [==============================] - 228s 509ms/step - loss: 1.5174 - accuracy: 0.4161 - val_loss: 1.4557 - val_accuracy: 0.4512
Epoch 4/50
448/448 [==============================] - 22s 49ms/step - loss: 1.4415 - accuracy: 0.4482 - val_loss: 1.3857 - val_accuracy: 0.4706
Epoch 5/50
448/448 [==============================] - 20s 45ms/step - loss: 1.3823 - accuracy: 0.4719 - val_loss: 1.3366 - val_accuracy: 0.4925
Epoch 6/50
448/448 [==============================] - 361s 808ms/step - loss: 1.3324 - accuracy: 0.4947 - val_loss: 1.3043 - val_accuracy: 0.5064
Epoch 7/50
448/448 [==============================] - 277s 620ms/step - loss: 1.2899 - accuracy: 0.5116 - val_loss: 1.2709 - val_accur

In [9]:
# save model structure in json file
model_json = emotion_model.to_json()
with open("emotion_model.json", "w") as json_file:
    json_file.write(model_json)

In [10]:
# save trained model weight in .h5 file
emotion_model.save_weights('emotion_model.h5')